**Important**: You don't need to execute this notebook if you have already downloaded the data using the notebook `00-download-data-archive.ipynb`.

In [15]:
import requests
import pandas as pd
import time
import os
from tqdm import tqdm

Define some parameters:
- The time to wait between the calls to the API
- The output file location and name
- The query / filter to use
- The API URL

In [16]:
wait_time_between_requests = 1 # 1 second as requested in the API documentation (https://xeno-canto.org/explore/api)
file_save_location = "../data/raw/xeno-canto.parquet"
audio_files_dir = "../data/raw/audio/xeno_canto/"

query = 'area:europe+len:"5-20"+q:">D"+grp:birds' # query for european birds with length 5-20 seconds and quality A, B or C (the three best qualities)
api_url = f"https://xeno-canto.org/api/2/recordings?query={query}&page="
recordings_to_ignore = ["692343", "545395", "394335", "351080", "473934", "471533", "827119", "471258", "567174", "672135"] # recordings with an invalid / missing audio file url

In [3]:
if not os.path.exists(audio_files_dir):
    os.makedirs(audio_files_dir)

Define the CSV columns

In [7]:
all_data = []
csv_headers = ["id", "gen", "sp", "ssp", "group", "en", "rec", "cnt", "loc", 
               "lat", "lng", "type", "sex", "stage", "method", "url", 
               "file", "file-name", "sono_small", "sono_med", "sono_large", 
               "sono_full", "osci_small", "osci_med", "osci_large", "lic", 
               "q", "length", "time", "date", "uploaded", "also", "rmk", 
               "bird-seen", "animal-seen", "playback-used", "temperature", 
               "regnr", "auto", "dvc", "mic", "smp"]

Define the function to get the data from the API

In [8]:
def get_data_from_api(page):
    response = requests.get(api_url + str(page))
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching data from page {page}: {response.status_code}")
        return None

Define the function to save the data to a CSV file

In [17]:
def save_to_parquet(data, filename):
    # Convert the list of dictionaries to a pandas DataFrame
    df = pd.DataFrame(data)
    # Save the DataFrame to a Parquet file
    df.to_parquet(filename, engine='pyarrow', index=False)
    print(f"Data saved to {filename}")

Define the function to save the audio files

In [10]:
def download_audio_file(file_url, file_name):
    try:
        audio_response = requests.get(file_url, stream=True)
        if audio_response.status_code == 200:
            # save the audio file to the specified directory
            file_path = os.path.join(audio_files_dir, file_name)

            with open(file_path, 'wb') as audio_file:
                for chunk in audio_response.iter_content(chunk_size=1024):
                    if chunk:
                        audio_file.write(chunk)
        else:
            print(f"Failed to download {file_name}: {audio_response.status_code}")
    except Exception as e:
        print(f"Error downloading {file_name}: {e}")

Loop through all pages and save all data into a list

In [11]:
# fetch the first page to get total pages
first_page_data = get_data_from_api(1)
if first_page_data:
    num_pages = first_page_data['numPages']
    print(f"Total pages to scrape: {num_pages}")

    # loop through all pages
    for page in tqdm(range(1, num_pages + 1)):
        print(f"Scraping page {page}...")
        
        # get the data from the current page
        page_data = get_data_from_api(page)
        if page_data and "recordings" in page_data:
            # extract relevant fields from each recording
            for recording in page_data["recordings"]:
                if recording.get("id") in recordings_to_ignore:
                    continue
                
                recording_data = {
                    "id": recording.get("id"),
                    "gen": recording.get("gen"),
                    "sp": recording.get("sp"),
                    "ssp": recording.get("ssp"),
                    "group": recording.get("group"),
                    "en": recording.get("en"),
                    "rec": recording.get("rec"),
                    "cnt": recording.get("cnt"),
                    "loc": recording.get("loc"),
                    "lat": recording.get("lat"),
                    "lng": recording.get("lng"),
                    "type": recording.get("type"),
                    "sex": recording.get("sex"),
                    "stage": recording.get("stage"),
                    "method": recording.get("method"),
                    "url": recording.get("url"),
                    "file": recording.get("file"),
                    "file-name": recording.get("file-name"),
                    "sono_small": recording["sono"].get("small") if "sono" in recording else "",
                    "sono_med": recording["sono"].get("med") if "sono" in recording else "",
                    "sono_large": recording["sono"].get("large") if "sono" in recording else "",
                    "sono_full": recording["sono"].get("full") if "sono" in recording else "",
                    "osci_small": recording["osci"].get("small") if "osci" in recording else "",
                    "osci_med": recording["osci"].get("med") if "osci" in recording else "",
                    "osci_large": recording["osci"].get("large") if "osci" in recording else "",
                    "lic": recording.get("lic"),
                    "q": recording.get("q"),
                    "length": recording.get("length"),
                    "time": recording.get("time"),
                    "date": recording.get("date"),
                    "uploaded": recording.get("uploaded"),
                    "also": ", ".join(recording.get("also", [])),
                    "rmk": recording.get("rmk"),
                    "bird-seen": recording.get("bird-seen"),
                    "animal-seen": recording.get("animal-seen"),
                    "playback-used": recording.get("playback-used"),
                    "temperature": recording.get("temp", ""),
                    "regnr": recording.get("regnr", ""),
                    "auto": recording.get("auto"),
                    "dvc": recording.get("dvc"),
                    "mic": recording.get("mic"),
                    "smp": recording.get("smp")
                }
                all_data.append(recording_data)

        # wait for 1 second before making the next request to not overload the server
        time.sleep(wait_time_between_requests)

Total pages to scrape: 179


  0%|          | 0/179 [00:00<?, ?it/s]

Scraping page 1...


  1%|          | 1/179 [00:04<13:48,  4.66s/it]

Scraping page 2...


...

 99%|█████████▉| 177/179 [12:47<00:10,  5.27s/it]

Scraping page 178...


 99%|█████████▉| 178/179 [12:52<00:05,  5.20s/it]

Scraping page 179...


100%|██████████| 179/179 [12:57<00:00,  4.34s/it]


Export the scraped data to a CSV file

In [18]:
print(f"Total recordings scraped: {len(all_data)}")
save_to_parquet(all_data, file_save_location)

Total recordings scraped: 89489
Data saved to ../data/raw/xeno-canto.parquet


Also download all the audio files to a folder. **Note**: This takes a long time (2-3 hours) and a lot of space (~100GB).

In [37]:
for recording_data in tqdm(all_data):
    audio_url = recording_data["file"]
    original_audio_file_name = recording_data["file-name"]
    file_extension = original_audio_file_name.split(".")[-1]
    new_audio_file_name = f"{recording_data['id']}.{file_extension}"

    download_audio_file(audio_url, new_audio_file_name)

 26%|██▌       | 9985/39083 [43:23<13:26:04,  1.66s/it]

Failed to download 692343.mp3: 502


 35%|███▌      | 13721/39083 [58:49<23:18:17,  3.31s/it]

Error downloading 545395.mp3: HTTPSConnectionPool(host='xeno-canto.org', port=443): Max retries exceeded with url: /545395/download (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f3bfc547340>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))


 65%|██████▍   | 25222/39083 [1:43:42<22:07, 10.44it/s]   

...

 81%|████████  | 31642/39083 [2:09:59<30:26,  4.07it/s]  

Failed to download 567174.mp3: 500


 86%|████████▌ | 33461/39083 [2:17:04<19:08,  4.89it/s]  

Error downloading 672135.: Invalid URL '': No scheme supplied. Perhaps you meant https://?


100%|██████████| 39083/39083 [2:39:18<00:00,  4.09it/s]
